Clase 9
======


Revisión de tarea optativa
--------------

1. Computa FBA de la red de *Escherichia coli* usando los valores por defecto de consumo de glucos y oxigeno.
2. Realiza FVA para la solución anterior.
3. Computa 100 muestras aleatorias del espacio de soluciones alternativas del punto 1, y luego calcula el valor promedio de todas sus reacciones.
4. Grafica los resultados usando ```escher```. En este caso, para cada flujo se debe calcular el valor promedio de las 100 muestras.

In [1]:
# 1. Calcular los flujos de E coli usando FBA
import cobra.test
model = cobra.test.create_test_model("ecoli")

In [2]:
# 3. Flux sampling
from cobra.sampling import sample 
s=sample(model,100) 


In [3]:
# Para obtener el promedio de todas las reacciones
meanFluxes=s.mean(axis=0)

In [4]:
import escher
from escher import Builder
# VISUALIZATION OF RESULTS
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)
builder.reaction_data = meanFluxes
builder

Builder(reaction_data={'DM_4crsol_c': 2.767893225024843e-06, 'DM_5drib_c': 0.004521004913532181, 'DM_aacald_c'…

Descargar modelos desde BiGG
-------------------

Hoy exploraremos el repositorio de modelos [BiGG](http://bigg.ucsd.edu/). 


Ejercicio
------------

Descarga el modelo de *Bacillus subtilis* iYO844 en formato SBML.

In [12]:
# Cargamos el modelo de Bacillus subtilis
bacilusModel=cobra.io.read_sbml_model("iYO844.xml")

In [13]:
# Información general del modelo
bacilusModel

Name,iYO844
Memory address,0x07f931effd410
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [14]:
# Reacciones de consumo de sustratos desde el medio de cultivo
bacilusModel.medium

{'EX_fe3_e': 999999.0,
 'EX_glc__D_e': 1.7,
 'EX_ca2_e': 999999.0,
 'EX_so4_e': 5.0,
 'EX_co2_e': 999999.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_mg2_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_nh4_e': 5.0,
 'EX_o2_e': 999999.0,
 'EX_pi_e': 5.0}

In [15]:
# Simular los flujos con FBA
solution=bacilusModel.optimize()

In [16]:
# Resumen de la solucion
bacilusModel.summary()

Especificando restricciones adicionales
---------------------------------------


Supón que queremos asegurarnos que dos reaccioens tengan el mismo flujo en nuestro modelo. Podemos agregar este criterio como una restricción a nuestro modelo a través de la siguiente sintaxis:

```python
same_flux = model.problem.Constraint(
    model.reactions.FBA.flux_expression - model.reactions.NH4t.flux_expression,
    lb=0,
    ub=0)
model.add_cons_vars(same_flux)
```

Ejercicio
------------

En el modelo de *Bacillus subtilis* especifica que el consumo de oxygeno 

1. sea igual al consumo de glucosa. 
2. sea 3 veces el consumo de glucosa


In [28]:
bacilusModel=cobra.io.read_sbml_model("iYO844.xml")
same_flux = bacilusModel.problem.Constraint(
    bacilusModel.reactions.EX_glc__D_e.flux_expression - bacilusModel.reactions.EX_o2_e.flux_expression,
    lb=0,
    ub=0)
bacilusModel.add_cons_vars(same_flux)
solution=bacilusModel.optimize()
bacilusModel.summary()

In [29]:
bacilusModel=cobra.io.read_sbml_model("iYO844.xml")
same_flux = bacilusModel.problem.Constraint(
    3*bacilusModel.reactions.EX_glc__D_e.flux_expression - bacilusModel.reactions.EX_o2_e.flux_expression,
    lb=0,
    ub=0)
bacilusModel.add_cons_vars(same_flux)
solution=bacilusModel.optimize()
bacilusModel.summary()

Especificando variables adicionales
---------------------------------------

Supón que queremos medir las diferencia entre el flujo de dos reacciones. Para esto podemos crear una variable adicional ```diff```.


```python
difference = model.problem.Variable('difference')
```

y luego definirla como la diferencia entre las dos reacciones de interes, por ejemplo:

```python
constraint = model.problem.Constraint(
    model.reactions.EX_glc__D_e.flux_expression -
    model.reactions.EX_nh4_e.flux_expression - difference,
    lb=0,
    ub=0)
model.add_cons_vars([difference, constraint])
```


Ejercicio
------------

En el modelo de *Bacillus subtilis* define una varibale que mida la diferencia entre los flujos de consumo de glucosa y nh4_e. 

In [37]:

difference = bacilusModel.problem.Variable('difference')
constraint = bacilusModel.problem.Constraint(
    bacilusModel.reactions.EX_glc__D_e.flux_expression -
    bacilusModel.reactions.EX_nh4_e.flux_expression - difference,
    lb=0,
    ub=0)
bacilusModel.add_cons_vars([difference, constraint])

In [39]:
solution=bacilusModel.optimize()
bacilusModel.solver.variables.difference.primal

-0.8632565053852966

In [40]:
solution["EX_glc__D_e"]-solution["EX_nh4_e"]

-0.8632565053852904